In [3]:
import numpy as np 
import pandas as pd
import os
violence = os.listdir('/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence')
nonviolence = os.listdir('/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence')

In [4]:
violence_path = [os.path.join('/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence',name) for name in violence]
nonviolence_path = [os.path.join('/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence',name) for name in nonviolence]

In [5]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
pretrained_model = InceptionV3()
pretrained_model = Model(inputs=pretrained_model.input,outputs=pretrained_model.layers[-2].output)

2024-03-13 12:46:34.605034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 12:46:34.605161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 12:46:34.754197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [15]:
def feature_extractor(frame):
    img = np.expand_dims(frame, axis=0)
    feature_vector = pretrained_model.predict(img, verbose=0)
    return feature_vector

In [16]:
def frames_extraction(video_path, SEQUENCE_LENGTH=16, IMAGE_WIDTH=299, IMAGE_HEIGHT=299, total_video=0):
    all_video_features = []
    for pos in tqdm(range(total_video)):
        frames_list = []
        video_reader = cv2.VideoCapture(video_path[pos])
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)
        for frame_counter in range(SEQUENCE_LENGTH):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            success, frame = video_reader.read()
            if not success:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            resized_frame = cv2.resize(frame_rgb, (IMAGE_HEIGHT, IMAGE_WIDTH))
            normalized_frame = resized_frame / 255
            features = feature_extractor(normalized_frame)
            frames_list.append(features)
        all_video_features.append(frames_list)
        video_reader.release()
    return np.array(all_video_features)

In [19]:
import cv2
violence_features = frames_extraction(violence_path[:200],total_video=len(violence_path[:200]))
non_violence_features = frames_extraction(nonviolence_path[:200],total_video=len(nonviolence_path[:200]))

100%|██████████| 200/200 [04:53<00:00,  1.47s/it]


In [20]:
np.save('violence_features.npy',violence_features)
np.save('non_violence_features.npy',non_violence_features)

In [21]:
non_violence_data = np.load('non_violence_features.npy')
violence_data = np.load('violence_features.npy')

In [22]:
from keras.models import Sequential
from keras.layers import LSTM, Dense,Bidirectional,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split

In [23]:
violence_labels = np.zeros(len(violence_data))
nonviolence_labels = np.ones(len(non_violence_data))

In [24]:

X = np.concatenate([violence_data, non_violence_data], axis=0)
y = np.concatenate([violence_labels, nonviolence_labels], axis=0)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [26]:

X_train_reshaped = X_train.reshape((X_train.shape[0], 16, 2048))
X_test_reshaped = X_test.reshape((X_test.shape[0], 16, 2048))

In [30]:
from tensorflow.keras import Input
inputs = Input(shape=(16, 2048))
x = Bidirectional(LSTM(200, return_sequences=True))(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(100))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

In [31]:
model = Model(inputs=inputs, outputs=outputs)

In [32]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 16, 2048)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 16, 400)        │     3,598,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_94          │ (None, 16, 400)        │         1,600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 400)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 200)            │       400,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_95          │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,042,001 (15.42 MB)

 Trainable params: 4,040,801 (15.41 MB)

 Non-trainable params: 1,200 (4.69 KB)

In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_reshaped,y_train,validation_data=(X_test_reshaped,y_test),epochs=5,batch_size=32)

Epoch 1/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.8675 - loss: 0.3545 - val_accuracy: 0.6750 - val_loss: 0.5508
Epoch 2/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9694 - loss: 0.1023 - val_accuracy: 0.9083 - val_loss: 0.4050
Epoch 3/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9851 - loss: 0.0540 - val_accuracy: 0.9000 - val_loss: 0.3854
Epoch 4/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9735 - loss: 0.0464 - val_accuracy: 0.9417 - val_loss: 0.3275
Epoch 5/5
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9984 - loss: 0.0079 - val_accuracy: 0.9583 - val_loss: 0.2811
